In [ ]:
import numpy as np
import pandas as pd
import h5py
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam as Adam
from keras.optimizers import SGD
from keras.utils import to_categorical

In [ ]:
# each image is represented by a 900-cell array
x = np.array(h5py.File('./datasets/train_pixels_tenK.hdf5', 'r')['train_preprocessed'])
y = np.array(pd.read_csv('./datasets/train_tenK_pixels_labels.csv', usecols=['Category']))

In [ ]:
reshaped_x = np.reshape(x, (-1, 30, 30, 1))

In [ ]:
# # train_y_list = np.array(pd.read_csv('train_labels.csv', usecols=['Category']))
label_array = [label[0] for label in y]

dict = {}
for label in label_array:
    if label not in dict:
        dict[label] = True
        
labels = list(dict.keys())

dict2 = {}
for i in range(0, len(labels)):
    dict2[labels[i]] = i  

y2 = []
for label in label_array:
    y2.append(dict2[label])

num_class = 31

categorical_y = to_categorical(y2, num_classes=31)
print(categorical_y)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(reshaped_x, categorical_y, test_size=0.4, shuffle=False)
test_x, valid_x, test_y, valid_y = train_test_split(test_x, test_y, test_size=0.5, shuffle=False)

In [ ]:
def cnn_model():
    # define sequential model
    model = Sequential()
    
    # input layer

    # filters detect the patterns on each image
    # Some filters detect edges, some circles, some corners and so on.
    # The deeper the network is, the more sophisticated the filters become
    # A filter can be thought of as a small matrix for which we decide how many rows and columns it has
    # Values within the matrix are initialized with random numbers
    # Let's say our matrix is 3 x 3
    # When this convolutional layer receives input, the filter will slide over each 3 x 3 set of pixels on the input image
    # This sliding is referred to as convolve

    num_filter = 128  # number of convolution filters to use
    num_row = 5  # number of rows in each convolution kernel 
    num_col = 5  # number of columns in each convolution kernel

    depth = 1
    width = 30
    height = 30

    model.add(Conv2D(num_filter, 
                     kernel_size=(num_row, num_col),
                     activation='relu',
                     input_shape=[width, height, depth],
                     strides=1,  # number units to shift for the next convolve
                     padding='same'))

    # After each conv layer, it is convention to apply a nonlinear layer (or activation layer) immediately afterward.
    # The purpose of this layer is to introduce nonlinearity to a system that basically has just been computing linear operations during the conv layers.
    # In the past, nonlinear functions like tanh and sigmoid were used, but researchers found out that ReLU layers work far better because the network is able to train a lot faster without making a significant difference to the accuracy. 
    # It also helps to alleviate the vanishing gradient problem, which is the issue where the lower layers of the network train very slowly because the gradient decreases exponentially through the layers . 
    # The ReLU layer applies the function f(x) = max(0, x) to all of the values in the input volume. In basic terms, this layer changes all the negative activations to 0.
    # This layer increases the nonlinear properties of the model and the overall network without affecting the receptive fields of the conv layer.
    model.add(LeakyReLU(alpha=0.3))

    # Normalize the activations of the previous layer at each batch, i.e. applies a transformation that maintains the
    # mean activation close to 0 and the activation standard deviation close to 1. 
    model.add(BatchNormalization(axis=-1))
    
    # 1st hidden layer
    model.add(Conv2D(num_filter, kernel_size=(num_row, num_col), strides=1, padding='same'))
    model.add(LeakyReLU(alpha=0.3))
    # Max Pooling: looks at the 2 neighboring pixels and picks the maximal value
    # Pooling is used to reduce computational cost, and to some extent also reduce overfitting
    # Combining convolutional and pooling layers, CNN are able to combine local features and learn more global features of the image.
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(BatchNormalization(axis=-1))

    # 2nd hidden layer
    model.add(Conv2D(num_filter*2, kernel_size=(num_row, num_col), strides=1, padding='same'))
    model.add(LeakyReLU(alpha=0.3))
    # pool
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    # 3rd hidden layer
    model.add(Conv2D(num_filter*2, kernel_size=(num_row, num_col), strides=1, padding='same'))
    model.add(LeakyReLU(alpha=0.3))
    # pool
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
    # The Flatten layer is use to convert the final feature maps into a one single 1D vector. 
    # This flattening step is needed so that you can make use of fully connected layers after some convolutional/maxpool layers. 
    # It combines all the found local features of the previous convolutional layers.
    model.add(Flatten())
    model.add(BatchNormalization(axis=-1))
    # Fully connected
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.3))
    model.add(BatchNormalization(axis=-1))
    # Dropout is a regularization method, where a proportion of nodes in the layer are randomly ignored for each training sample. 
    # This drops randomly a propotion of the network and forces the network to learn features in a distributed way. 
    # This technique also improves generalization and reduces the overfitting.
    model.add(Dropout(0.4))
    
    # output layer
    model.add(Dense(num_class, activation='softmax'))

    model.summary()

    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [ ]:

def cnn_model2():
    model = Sequential()
    kernel_size = (5, 5)
    model.add(Conv2D(64, kernel_size, padding='same', input_shape=[100, 100, 1], name='block1_conv1'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(64, kernel_size, padding='same', name='block1_conv2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

    model.add(Conv2D(128, kernel_size, padding='same', name='block2_conv1'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(128, kernel_size, padding='same', name='block2_conv2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

    model.add(Conv2D(256, kernel_size, padding='same', name='block3_conv1'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(256, kernel_size, padding='same', name='block3_conv2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(256, kernel_size, padding='same', name='block3_conv3'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(256, kernel_size, padding='same', name='block3_conv4'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

    model.add(Conv2D(512, kernel_size, padding='same', name='block4_conv1'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, kernel_size, padding='same', name='block4_conv2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, kernel_size, padding='same', name='block4_conv3'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, kernel_size, padding='same', name='block4_conv4'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

    model.add(Conv2D(512, kernel_size, padding='same', name='block5_conv1'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, kernel_size, padding='same', name='block5_conv2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, kernel_size, padding='same', name='block5_conv3'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, kernel_size, padding='same', name='block5_conv4'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Flatten())

    model.add(Dense(4096))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(4096, name='fc2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(31))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('softmax'))

    sgd = SGD(lr=0.0005, decay=0, nesterov=True)

    model.summary()

    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
BATCH_NORM = True
model = cnn_model()

epoch = 100
batch_size = 512

model.fit(train_x, train_y, 
        validation_data=(valid_x, valid_y),
        shuffle=True, 
        epochs=epoch, 
        batch_size=batch_size, 
        verbose=2)



In [ ]:
final_x  = np.array(h5py.File('./datasets/test_pixels_tenK.hdf5', 'r')['test_preprocessed'])

In [ ]:
model.save_weights('cnn_model_weights.hdf5')

# Model prediction on testing data
reshaped_final_x = np.reshape(final_x, (-1, 30, 30, 1))
best = model.predict(reshaped_final_x, batch_size=batch_size)
best = np.argmax(best, axis=1) 

# # Remap the indice of one hot encoded labels to its original label:
remap = lambda x: labels[x]

best = best.tolist()     

best = [remap(indice) for indice in best]
print(best)

In [ ]:
df = pd.DataFrame(np.array(best), columns=['Category'])
df.to_csv('kaggle_test3.csv', index_label='Id')